In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import csv
import warnings
warnings.simplefilter('ignore')

In [3]:
#sub_list = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
filename = '/content/drive/MyDrive/Colab Notebooks/preprocessed_data/2class/2_df_sub16.csv'
df = pd.read_csv(filename)
df = df.dropna(axis=0)
df

,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label
0,0.030533,0.016342,0.988770,33.376709,6.391907,0.0
1,0.044449,-0.008194,0.972366,33.363129,6.391907,0.0
2,0.050903,-0.026413,0.966263,33.376709,6.404114,0.0
3,0.051590,0.002747,1.010132,33.379761,6.398010,0.0
4,0.052917,0.003113,0.963593,33.331451,6.410217,0.0
...,...,...,...,...,...,...
1554696,-0.068298,0.002060,2.163315,34.381134,-2.624512,0.0
1554697,-0.068893,0.010025,2.179337,34.368866,-2.626038,0.0
1554698,-0.070587,0.017075,2.175140,34.368866,-2.619934,0.0
1554699,-0.066422,0.007828,2.169800,34.355103,-2.870178,0.0


In [4]:
X = df.drop('w_label', axis = 1)
y = df.w_label

In [5]:
X

,c_ecg,c_emg,c_eda,c_temp,c_resp
0,0.030533,0.016342,0.988770,33.376709,6.391907
1,0.044449,-0.008194,0.972366,33.363129,6.391907
2,0.050903,-0.026413,0.966263,33.376709,6.404114
3,0.051590,0.002747,1.010132,33.379761,6.398010
4,0.052917,0.003113,0.963593,33.331451,6.410217
...,...,...,...,...,...
1554696,-0.068298,0.002060,2.163315,34.381134,-2.624512
1554697,-0.068893,0.010025,2.179337,34.368866,-2.626038
1554698,-0.070587,0.017075,2.175140,34.368866,-2.619934
1554699,-0.066422,0.007828,2.169800,34.355103,-2.870178


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [8]:
check_for_nan = df.isnull().values.any()
print (check_for_nan)

False


In [9]:
import sklearn as sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report

In [10]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [11]:
pd.Series(y).value_counts()

0.0    1083600
1.0     471101
Name: w_label, dtype: int64

In [12]:
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

# **Random Forest**

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
forest = RandomForestClassifier(n_estimators = 50, min_samples_leaf = 5)

In [15]:
sfold = StratifiedKFold(n_splits = 10, random_state = 0, shuffle = True)
cross_validate(forest, x_train, y_train, 
               scoring=['accuracy', 'f1'], 
               return_train_score=True,
               cv = sfold)

{'fit_time': array([22.76251578, 18.01239228, 20.67646646, 21.89962435, 21.07253551,
        21.44671369, 22.92326975, 22.22553873, 23.63961792, 22.2940731 ]),
 'score_time': array([0.27847409, 0.27069998, 0.26764727, 0.28852701, 0.26682711,
        0.27934265, 0.27727771, 0.26685095, 0.285182  , 0.27676368]),
 'test_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'test_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

# **Random Forest(SMOTE-ENN)**

In [16]:
from imblearn.combine import SMOTEENN

SMOTEENN = SMOTEENN(random_state=0)
x_train_over,y_train_over = SMOTEENN.fit_resample(x_train,y_train)

print('SMOTEENN 적용 전 학습용 피처/레이블 데이터 세트: ', x_train.shape, y_train.shape)
print('SMOTEENN 적용 후 학습용 피처/레이블 데이터 세트: ', x_train_over.shape, y_train_over.shape)
print('SMOTEENN 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTEENN 적용 전 학습용 피처/레이블 데이터 세트:  (932820, 5) (932820,)
SMOTEENN 적용 후 학습용 피처/레이블 데이터 세트:  (1300372, 5) (1300372,)
SMOTEENN 적용 후 레이블 값 분포: 
 0.0    650186
1.0    650186
Name: w_label, dtype: int64


In [17]:
sfold = StratifiedKFold(n_splits = 10, random_state = 0, shuffle = True)
cross_validate(forest, x_train_over, y_train_over, 
               scoring=['accuracy', 'f1'], 
               return_train_score=True,
               cv = sfold)

{'fit_time': array([32.57435918, 42.00226998, 31.19308662, 37.4231267 , 34.81610727,
        33.63686156, 32.14463949, 34.01538205, 32.85887599, 37.21910596]),
 'score_time': array([0.32347035, 0.35105324, 0.31310463, 0.3312943 , 0.31948805,
        0.31273031, 0.31751513, 0.31521487, 0.32805252, 0.33684683]),
 'test_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'test_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_accuracy': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'train_f1': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}